# <center>2D Poisson Equation Solver</center>

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import spsolve
import plotly.graph_objects as go
import numpy.ma as ma
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import diags, bmat
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix, csc_matrix

### <center>Define the boundary conditions</center>

In [13]:
def boundary (N):
    U = np.zeros((N, N))
    U[0, 1:] = -1
    U[1:N-1, 0] = 1
    
    return U

In [14]:
def show_boundary(U, N):
    U = np.array([[np.nan if i + j == N else U[i, j] for j in range(N)] for i in range(N)])
    # Create X, Y, Z arrays for scatter plot
    x, y, z = [], [], []
    for i in range(N):
        for j in range(N):
            if not ma.is_masked(U[i, j]):  # Check if the value is not masked
                x.append(i / (N - 1))  # Normalize x
                y.append(j / (N - 1))  # Normalize y
                z.append(U[i, j])  # Normalize z to range 0 to 1

    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                       marker=dict(size=4, color=z, colorscale='Viridis', opacity=0.8))])

    # Update layout
    fig.update_layout(
        title='3D Scatter Plot of Defined Values in U (Normalized to 1x1x1 Cube)',
        scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='U Value',
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=1)),
        autosize=False,
        width=700,
        height=700,
        margin=dict(l=65, r=50, b=65, t=90)
    )

    # Show the plot
    fig.show()


T is the tridiagonal matrix of the form:
\begin{equation} T=\left(\begin{array}{}
-4&1&0&0&.&.&.\\
1&-4&1&0&0&.&.\\
.&.&.&.&.&.&.\\
.&.&.&0&1&-4&1\\
.&.&.&.&0&1&-4\\
\end{array}\right).
\end{equation}

A is an $(N-1)^2\times(N-1)^2$  matrix made up of the following block tridiagonal structure
\begin{equation}\left(\begin{array}{ccccccc}
T&I&0&0&.&.&.\\
I&T&I&0&0&.&.\\
.&.&.&.&.&.&.\\
.&.&.&0&I&T&I\\
.&.&.&.&0&I&T\\
\end{array}\right),
\end{equation}

In [15]:
def A_matrix(N):
    I = np.eye(N)  # Identity matrix
    T = np.zeros((N, N)) - 4 * np.eye(N)
    for i in range(0, N-1):
        T[i, i+1] = 1
        T[i+1, i] = 1
    
    A = np.zeros((N**2, N**2))
    for i in range(N):
        for j in range(N):
            if i == j:
                A[i*N:(i+1)*N, j*N:(j+1)*N] = T
            elif abs(i-j) == 1:
                A[i*N:(i+1)*N, j*N:(j+1)*N] = I
    
    A_csr = csr_matrix(A)
    return A_csr

# f function inside the domain

In [16]:
def f_function(N, c=1):
    f = c*np.ones((N, N))
    f[0, 1:N] = 0
    f[1:N, 0] = 0
    f = np.array([[0 if i+j >= N else f[i, j] for j in range(N)] for i in range(N)])
    return f

In [17]:
def solution_u (U, f, A, N):

    h = 1/N
    f *= h**2
    U *= h**2
    f_flat = f.flatten()
    U_flat = U.flatten()

    b = f_flat + U_flat
    U_solution = spsolve(A, b)
    return U_solution

In [18]:
def plot_result(U_solution, N):

    U_solution = U_solution.reshape(N, N)
    U_solution = np.array([[np.ma.masked if i+j > N else U_solution[i, j] for j in range(N)] for i in range(N)])
    
    # Create X, Y, Z arrays for scatter plot
    x, y, z = [], [], []
    max_z = np.nanmax(U_solution)  # Find the maximum z value
    for i in range(N):
        for j in range(N):
            x.append(i / (N - 1))  # Normalize x
            y.append(j / (N - 1))  # Normalize y
            z.append(U_solution[i, j] / max_z)  # Normalize z to range 0 to 2
    
    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                       marker=dict(size=4, color=z, colorscale='Viridis', opacity=0.8))])
    
    # Update layout
    fig.update_layout(
        title='3D Scatter Plot of Defined Values in U (Normalized to 1x1x2 Cube)',
        scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='U Value',
            aspectmode='manual',
            aspectratio=dict(x=1, y=1, z=1)),
        autosize=False,
        width=700,
        height=700,
        margin=dict(l=65, r=50, b=65, t=90)
    )


In [24]:
N = 50
U = boundary(N)
A = A_matrix(N)
F = f_function(N, c=0)
U_solution = solution_u(U, F, A, N)

In [25]:
plot_result(U_solution,N)

/var/folders/q6/nvh9wdyj25gd4w7jkh1h8w600000gn/T/ipykernel_9768/48553796.py:4: UserWarning:




In [26]:
U_solution = U_solution.reshape(N, N)
U_solution = np.array([[np.ma.masked if i+j > N else U_solution[i, j] for j in range(N)] for i in range(N)])

# Create X, Y, Z arrays for scatter plot
x, y, z = [], [], []
max_z = np.nanmax(U_solution)  # Find the maximum z value
for i in range(N):
    for j in range(N):
        x.append(i / (N - 1))  # Normalize x
        y.append(j / (N - 1))  # Normalize y
        z.append(U_solution[i, j] / max_z)  # Normalize z to range 0 to 2

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers',
                                   marker=dict(size=4, color=z, colorscale='Viridis', opacity=0.8))])

# Update layout
fig.update_layout(
    title='3D Scatter Plot for potential',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='U Value',
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1)),
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90)
)


/var/folders/q6/nvh9wdyj25gd4w7jkh1h8w600000gn/T/ipykernel_9768/1310612576.py:2: UserWarning:


